## Connect to Google Drive to access Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd /content/drive/MyDrive

In [ ]:
!pip install opendatasets
import opendatasets as od
import pandas as pd
od.download("https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database")

In [ ]:
!unzip '/content/drive/MyDrive/covid19-radiography-database/covid19-radiography-database.zip' -d '/content/drive/MyDrive/covid19-radiography-database'

In [ ]:
!ls "/content/drive/MyDrive/covid19-radiography-database/COVID-19_Radiography_Dataset"

## Create Training And Testing Dataset

In [ ]:
!mkdir /content/drive/MyDrive/Xray_train_data

In [ ]:
!cp -R "/content/drive/MyDrive/covid19-radiography-database/COVID-19_Radiography_Dataset/COVID/images" "/content/drive/MyDrive/Xray_train_data/COVID"

In [ ]:
!ls -1 "/content/drive/MyDrive/Xray_train_data/COVID"| wc -l

In [ ]:
!mkdir "/content/drive/MyDrive/Xray_train_data/NORMAL"

In [ ]:
import pandas as pd
PATH_TO_METADATA = "/content/drive/MyDrive/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal.metadata.xlsx"
df = pd.read_excel(PATH_TO_METADATA)
df.head()

In [ ]:
import os
import shutil
cnt = 0
for (i, row) in df.iterrows():
    if (cnt < 3616):
        filename = row["FILE NAME"].lower().capitalize() + "." + row["FORMAT"].lower()
        image_path = os.path.join("/content/drive/MyDrive/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images", filename)
        image_copy_path = os.path.join("/content/drive/MyDrive/Xray_train_data/NORMAL", filename)
        shutil.copy2(image_path, image_copy_path)
        cnt += 1

In [ ]:
print(cnt)

In [ ]:
!ls "/content/drive/MyDrive/Xray_train_data"

In [ ]:
IMAGE_SIZE = (256, 256)
IMAGE_SHAPE = IMAGE_SIZE + (3,)

In [ ]:
train_data_dir= "/content/drive/MyDrive/Xray_train_data"

In [ ]:
test_data_dir= "/content/drive/MyDrive/Xray_train_data"

##Importing The Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_height, img_width= IMAGE_SIZE
batch_size=16
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,shear_range=0.2,validation_split=0.3)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation')

# VGG16

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [ ]:
vgg = VGG16(include_top=False,input_shape=(256,256,3))

In [ ]:
vgg.summary()

In [ ]:
for layer in vgg.layers:
  layer.trainable=False

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
output = Dense(1, activation='sigmoid')(x)

In [ ]:
vgg16 = Model(vgg.input,output)

In [ ]:
vgg16.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
vgg16.fit(train_generator,validation_data=validation_generator,epochs=5)

In [ ]:
probabilities = vgg16.predict(generator=validation_generator)
probabilities

In [ ]:
y_true = validation_generator.classes
y_true

In [ ]:
y_pred = probabilities > 0.5
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix
cf_matrix = confusion_matrix(y_true, y_pred)
print(cf_matrix)

In [ ]:
import seaborn as sns
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True,
            fmt='.2%', cmap='Blues')

In [ ]:
import numpy as np

def calculate_metrics(confusion_matrix):
    # Extract values from the confusion matrix
    TP = confusion_matrix[1, 1]
    TN = confusion_matrix[0, 0]
    FP = confusion_matrix[0, 1]
    FN = confusion_matrix[1, 0]

    # Calculate accuracy
    accuracy = (TP + TN) / np.sum(confusion_matrix)

    # Calculate precision
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0

    # Calculate recall (sensitivity/TPR)
    recall = TP / (TP + FN) if (TP + FN) != 0 else 0

    # Calculate specificity
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0

    return accuracy, precision, recall, specificity

accuracy, precision, recall, specificity = calculate_metrics(cf_matrix)

print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'Specificity: {specificity:.2f}')


In [ ]:
vgg16.save("vgg16.h5")